In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

data = pd.read_csv(r"C:\Courses\DEPI\Slides\project depi\SuperStoreSales.csv")



In [2]:
print(data.head())


   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017    Second Class    DV-13045   
3       4  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   
4       5  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

   Postal Code Region       Product ID         Cat

In [3]:
print(data.isnull().sum())


Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
dtype: int64


In [4]:
data["Postal Code"] = data.groupby("City")["Postal Code"].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else None))


In [9]:
print(data.isnull().sum())


Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
dtype: int64


In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data["Sales"] = scaler.fit_transform(data[["Sales"]])


In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data["Sales"] = scaler.fit_transform(data[["Sales"]])


In [21]:
dummies = pd.get_dummies(
    data[["Ship Mode", "Segment", "Country", "City", "State", "Region", "Category", "Sub-Category"]], 
    drop_first=False
)
data = pd.concat([data, dummies], axis=1)


In [29]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()  
data["Ship Mode Encoded"] = encoder.fit_transform(data["Ship Mode"])


In [39]:
#data["Income_per_Hour"] = data["Total_Income"] / data["Working_Hours"]
data["Income_per_Hour"] = data.get("Total_Income", 0) / data.get("Working_Hours", 1)


In [43]:
data["Order Date"] = pd.to_datetime(data["Order Date"], format="%d/%m/%Y", errors='coerce')
data["Ship Date"] = pd.to_datetime(data["Ship Date"], format="%d/%m/%Y", errors='coerce')

data["Shipping_Duration"] = (data["Ship Date"] - data["Order Date"]).dt.days


In [45]:
data["Order Date"] = pd.to_datetime(data["Order Date"], format="%d/%m/%Y", errors='coerce')

data["Order_Year"] = data["Order Date"].dt.year
data["Order_Month"] = data["Order Date"].dt.month
data["Order_Day"] = data["Order Date"].dt.day
data["Order_Weekday"] = data["Order Date"].dt.weekday


In [33]:
data["Sales_per_Product"] = data.groupby("Product ID")["Sales"].transform("mean")


In [35]:
data["City_Segment"] = data["City"] + "_" + data["Segment"]
